# Introduction to Generative LLMs for Social Science Research

June Yang, CSDE and eScience Institute, Winter 2025

Hello! Welcome to the workshop on using LLMs for social science research.

For today's session, we will first go over the use cases of LLMs in social science research, and then dive deeper into two methods (chat completion and RAG) using the `langchain` package and an OpenAI model (gpt-3.5-turbo). As this field is quickly evolving, I encourage you to explore other models, especially the open-source ones, and think of your own research applications creatively! 

Proficiency in Python is not required, as coding is not the focus in this workshop, but basic understandings of writing functions and loops will be helpful. We will also be covering some relatively advanced topics in text analysis, so some prime knowledge in Text as Data will also be helpful. For a refresher, feel free to check out my previous tutorial on [Text as Data](https://colab.research.google.com/drive/1RWUytojPxMkMQs2pDo7EYKwMBRP76IUo?usp=sharing).


# Table of Contents

- [Resources and Materials](#resources-and-materials)
- [What is a generative LLM?](#what-is-a-generative-llm)
- [The use of generative LLMs for the social sciences](#the-use-of-generative-llms-for-the-social-sciences)
- [Implementation using `LangChain`](#implementation-using-langchain)
  - [Introducting the `LangChain` package](#introducing-the-langchain-package)
  - [Case 1: Using Chat Completion for Text Annotation](#using-chat-completion-for-text-annotation)
    - [Models (LLM Wrappers)](#step-1-models-llm-wrappers)
    - [Prompts](#step-2-prompts)
    - [Chains](#step-3-chains)
    - [Validation](#step-4-validation)
    - [Model Selection](#model-selection)
  - [Case 2: Embeddings, RAG, and Vector Stores](#advanced-topic-embeddings-rag-and-vector-stores)
    - [What is word embedding](#what-is-word-embedding)
    - [What is RAG?](#what-is-rag)
      - [Differences between RAG and Fine-Tuning](#differences-between-rag-and-fine-tuning)
      - [How to implement RAG?](#how-to-implement-rag)
- [References](#references)

# Resources and Materials

- [DeepSeek](https://www.deepseek.com/en)
- [OpenAI API Reference](https://platform.openai.com/docs/api-reference)
- [LangChain Documentation](https://docs.langchain.com/docs/)
- [Hugging Face](https://huggingface.co)
- [Parlance Labs](https://parlance-labs.com/education/prompt_eng/berryman.html)
- [The landscape of LLMs and their openness](https://opening-up-chatgpt.github.io)
- [eScience SSEC tutorial](https://uw-ssec-tutorials.readthedocs.io/en/latest/SciPy2024/README.html) 




# What is a generative LLM?

Recommended reading: [BERT vs. GPT: A Tale of Two Transformers that Revolutionized NLP](https://medium.com/@prudhvithtavva/bert-vs-gpt-a-tale-of-two-transformers-that-revolutionized-nlp-11fff8e61984)

Generative vs. Discriminative Models

- Generative Models: These models learn to generate new data instances that resemble the training data. In language models, this means they can produce text, complete sentences, and engage in open-ended tasks like translation or summarization. Examples include probablistic generative models, neural generative models such as GPT (Generative Pre-trained Transformer), as well as hybrid models (e.g. topic guided transformers).
	
- Discriminative (or Classifier) Models: These models focus on making predictions or classifications based on input data. For example, they might classify sentiment, identify named entities, or categorize topics. Models like BERT (Bidirectional Encoder Representations from Transformers) are discriminative since they’re optimized for predicting specific labels given input text, such as in sentiment analysis or question answering.

# The use of generative LLMs for the social sciences

Reference: Large Language Models (LLMs) in Social Science Research Session I: Introduction

Joshua Cova and Luuk Schmitz, MPISS, 20-06-2024

How can LLMs help social science research?

- Modelling human behavior computationally 
  - Testing & (potentially) running experiments: Aher, Arriaga, and Kalai (2022); Dillion et al(2023)
  - Running surveys: Tjuatia et al.(2023)
  - Integreating social network interactions with LLMs: Jiang and Ferrara (2023)

- Simulating social relationships
  - Simulating social networks: Interactions between articifial agents: Park et al.(2023); Wang et al.(2024)
  - Game theoretical simulations: Akata et al.(2023)

- Interacting with human agents   
  - Chatbots for interviewing partiticipants: Chopra and Haaland (2023)
  
- Text annotation 
  - Zero-shot & few-shot text annotation: Törnberg (2023); Gilardi, Alizadeh, and Kubi (2023); Leek, Bischl, and Freier (2024)
  - Synthetic data generation: Laurer (2024)

LLMs can also help streamline the research pipeline more generally. 

- Spitballing ideas
- Data preparation
- Summazing texts (e.g., Interview data)
- Creative writing
- Conducting preliminary analyses 
  
See also Korinek (2023)

# Implementation using `LangChain`


### Introducing the `LangChain` package

[LangChain](https://docs.langchain.com/docs/) is an open-source Python framework that streamlines the development of applications powered by Large Language Models (LLMs). It offers a suite of tools and components that simplify the construction of LLM-centric applications, making it particularly useful for social scientists interested in leveraging generative AI for their research.

Key Features of LangChain:

- Prompt Templates: LangChain provides reusable templates that can be dynamically adjusted by inserting specific values, allowing for the generation of prompts based on dynamic resources. ￼

- Chains: These are sequences of components or actions linked together to process inputs and produce desired outputs. For example, a chain might involve retrieving data, processing it through an LLM, and then formatting the output. ￼
	
- Memory: This feature allows applications to remember previous interactions, enabling more contextually relevant responses in conversational AI applications. ￼
	
- Agents: Agents can perform actions based on user input, such as querying databases or calling APIs, and then use LLMs to generate responses based on the results. ￼
	
- Integration with External Data: LangChain facilitates the incorporation of external data sources, such as APIs and databases, enhancing the relevance and accuracy of generated responses. ￼


### Case 1: Using Chat Completion for Text Annotation

In this tutorial, we're going to replicate Törnberg (2023): [How to use Large Language Models for Text Analysis](https://arxiv.org/abs/2307.13106) by breaking the task into processes.

What is the paper doing?

This paper introduces the *WHAT* and *HOW* on using LLMs for text analysis, specifically focusing on the use of chat completion for text annotation. It offers an example of using the OpenAI API and model gpt-4 to annotate the level of populism in political speeches. Note that this is a creative and customized measurement that can be generated based on theorecial knowledge, traditionally done by human annotators. In the example below, we will be reproducing the paper, but not entirely using the OpenAI python library. Rather, we will mainly use the `LangChain` package to implement the same task.

Before we start, there are some important concepts to understand about APIs:

An Application Programming Interface (API) is like a waiter at a restaurant - it acts as an intermediary that takes requests and returns responses. APIs allow different software systems to communicate without needing to understand each other's internal workings.

Key concepts of APIs:

- They provide a standardized way to request services or data
- They define the methods and data formats for interaction
- They handle authentication and access control
- They often have usage limits and pricing tiers

For LLMs like OpenAI's GPT models or Anthropic's Claude, APIs allow developers to:

- Access Model Capabilities
  - Send text prompts and receive generated responses
  - Control various parameters like temperature (creativity) and max tokens
  - Access different model versions and capabilities
  
- Manage Resources
  - Track usage and costs
  - Handle rate limiting and quotas
  - Manage API keys and authentication
  
- Integration Options
  - Direct API calls for basic interactions
  - SDK (Software Development Kit) support for easier integration
  - Framework support through tools like LangChain
  
- Common LLM API Features
  - Authentication
    - API keys for secure access
    - Organization IDs for team management
  - Request Parameters
    - Model selection (e.g., GPT-4, GPT-3.5-turbo)
    - Temperature and other generation controls
    - System and user messages
    - Context and memory management

  - Response Handling
    - Generated text
    - Token usage statistics
    - Error messages and rate limit information

This API-based approach allows researchers and developers to leverage powerful LLM capabilities without needing to host or maintain the models themselves. For an example, see the [OpenAI API Reference](https://platform.openai.com/docs/api-reference/introduction), [playground](https://platform.openai.com/playground), and [pricing](https://openai.com/api/pricing/) page. Of course, with the lastest development of DeepSeek, hosting your own model has become an option as well (actually, a preferable option). 

We'll proceed with replicating the paper in the below sections.

#### Step 1. Models (LLM Wrappers)

In [41]:
# install packages in your environment:
# pip install langchain openai pandas scikit-learn dotenv

from dotenv import load_dotenv,find_dotenv
from langchain_community.llms import OpenAI
import pandas as pd
import os
import chardet
import time  

# optional: suppress warnings
import warnings
warnings.filterwarnings('ignore')
# Specifically for LangChain deprecation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [42]:
# Set up OpenAI API key
load_dotenv(find_dotenv())

# Initialize the LLM
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

# Test the LLM
print(llm.invoke("Hello, world!"))

Hello! How can I assist you today?


In [43]:
print(llm.model_name)

gpt-3.5-turbo


Dataset we are using: Hawkins, Kirk A., Rosario Aguilar, Erin Jenne, Bojana Kocijan, Cristóbal Rovira Kaltwasser, Bruno Castanho Silva. 2019. Global Populism Database: Populism Dataset for Leaders 1.0. [link](https://populism.byu.edu/data/2019%20-%20global%20populism%20database%20(guardian%20version))


The political texts are in different languages, and possibly follows different text encoding standards. We define a function on auto encoding below. Note that this is also a common step we see when working with different datasets.

In [44]:
def read_file_with_auto_encoding(file_path):
    # First, detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            text = file.read()
            # Remove BOM if present
            if text.startswith('\ufeff'):
                text = text[1:]
                
            # Find the actual speech content
            lines = text.split('\n')
            content_start = 0
            
            # First try: Look for quotation marks
            for i, line in enumerate(lines):
                if '"' in line or '"' in line:
                    content_start = i
                    break
            
            # Second try: Look for first substantial text after metadata
            if content_start == 0:
                empty_lines = 0
                for i, line in enumerate(lines):
                    if not line.strip():
                        empty_lines += 1
                    elif empty_lines >= 2:  # After finding 2+ empty lines, next non-empty is content
                        content_start = i
                        break
            
            # Third try: Look for first empty line (simpler fallback)
            if content_start == 0:
                for i, line in enumerate(lines):
                    if line.strip() == '' and i > 0:
                        content_start = i + 1
                        break
            
            # Get the content
            content = '\n'.join(lines[content_start:])
            
            # Clean up any leading/trailing whitespace
            content = content.strip()
            
            return content
            
    except UnicodeDecodeError:
        # Fallback to latin-1 if detection fails
        with open(file_path, 'r', encoding='latin-1') as file:
            text = file.read()
            if text.startswith('\ufeff'):
                text = text[1:]
            return text.strip()
    except Exception as e:
        print(f"Error reading file {file_path}: {str(e)}")
        return ""

We will now read all the speeches from the folder and store them in a dataframe.

In [45]:
# Path to the folder containing speech text files
folder_path = 'data/global-populism-dataset-zi/speeches_20220427/speeches_20220427'
# Initialize an empty list to store speech data
data = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        try:
            text = read_file_with_auto_encoding(file_path)
            speech_name = os.path.splitext(filename)[0]
            data.append({'speech_name': speech_name, 'text': text})
        except Exception as e:
            print(f"Failed to read file {filename}: {str(e)}")
            continue

# Create a DataFrame with the collected data
df = pd.DataFrame(data)
print("\nFirst few rows:")
print(df.head())


First few rows:
                              speech_name  \
0               Nicaragua_Ortega_Famous_1   
1                  France_Chirac_Famous_1   
2                   Serbia_Tadic_Famous_1   
3  Georgia_Margvelashvili_International_1   
4                       UK_Blair_Ribbon_3   

                                                text  
0  Primer Año del Gobierno del Pueblo\nPrimer Ani...  
1  "Déclaration aux Français de Monsieur Jacques ...  
2  Држава Србија је забринута што је изостала реа...  
3                                                     
4  A Cabinet Minister is sitting having a pub lun...  


In [46]:
# Print a sample of text from each row to verify encoding
print("\nSample of first 100 characters from each text:")
for idx, row in df.head().iterrows():
    print(f"\n{row['speech_name']}:")
    print(row['text'][:100])


Sample of first 100 characters from each text:

Nicaragua_Ortega_Famous_1:
Primer Año del Gobierno del Pueblo
Primer Aniversario del Gobierno de 
Reconciliación y Unidad Nacio

France_Chirac_Famous_1:
"Déclaration aux Français de Monsieur Jacques CHIRAC, Président de la République.

Palais de l'Elysé

Serbia_Tadic_Famous_1:
Држава Србија је забринута што је изостала реакција Уједињениих нација, УНМИК-а, поводом проглашења 

Georgia_Margvelashvili_International_1:


UK_Blair_Ribbon_3:
A Cabinet Minister is sitting having a pub lunch when a member of the public accosts him.  “I’ve bee


Each model has a token limit, which is the maximum number of tokens that can be processed in a single request. We will now define a function to count the number of tokens in a text using the `tiktoken` package. When using the API request, the `gpt-3.5-turbo` model we are using is the 16k variant, with a token limit of 16384. This includes the system prompt, user prompt, and response. For details, see OpenAI's documentation [here](https://platform.openai.com/docs/models#gpt-3-5-turbo).

In [47]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken

# Initialize the tokenizer for GPT-3.5-turbo
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=10000,  # Approximate number of tokens
    chunk_overlap=200,  # Some overlap to maintain context
    length_function=lambda text: len(encoding.encode(text)),  # Use the same token counter
    separators=["\n\n", "\n", " ", ""]  # Default separators
)

# Apply the splitter to each text in the DataFrame
df['chunks'] = df['text'].apply(lambda x: text_splitter.split_text(x))

# Print the first few rows to verify
print(df[['speech_name', 'chunks']].head())

                              speech_name  \
0               Nicaragua_Ortega_Famous_1   
1                  France_Chirac_Famous_1   
2                   Serbia_Tadic_Famous_1   
3  Georgia_Margvelashvili_International_1   
4                       UK_Blair_Ribbon_3   

                                              chunks  
0  [Primer Año del Gobierno del Pueblo\nPrimer An...  
1  ["Déclaration aux Français de Monsieur Jacques...  
2  [Држава Србија је забринута што је изостала ре...  
3                                                 []  
4  [A Cabinet Minister is sitting having a pub lu...  


In [48]:
# Get length of chunks list for each row
df['chunk_count'] = df['chunks'].apply(len)

# Show distribution of chunk counts
print("Chunk count distribution:")
print(df['chunk_count'].value_counts().sort_index())

# Show some example rows with their chunk counts
print("\nExample rows with chunk counts:")
print(df[['speech_name', 'chunk_count']].head())

# Get summary statistics
print("\nSummary statistics of chunk counts:")
print(df['chunk_count'].describe())

Chunk count distribution:
chunk_count
0    111
1    943
2     71
3     22
4      7
5      5
6      2
Name: count, dtype: int64

Example rows with chunk counts:
                              speech_name  chunk_count
0               Nicaragua_Ortega_Famous_1            2
1                  France_Chirac_Famous_1            1
2                   Serbia_Tadic_Famous_1            1
3  Georgia_Margvelashvili_International_1            0
4                       UK_Blair_Ribbon_3            1

Summary statistics of chunk counts:
count    1161.000000
mean        1.047373
std         0.629993
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         6.000000
Name: chunk_count, dtype: float64


Remove the zero-chunk rows.

In [49]:
# Remove rows where chunks list is empty
df_clean = df[df['chunks'].apply(len) > 0].copy()

# Print before/after stats
print(f"Original number of speeches: {len(df)}")
print(f"Number after removing empty chunks: {len(df_clean)}")
print(f"Removed {len(df) - len(df_clean)} speeches")

# Reset the index if needed
df_clean = df_clean.reset_index(drop=True)

Original number of speeches: 1161
Number after removing empty chunks: 1050
Removed 111 speeches


In [50]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   speech_name  1050 non-null   object
 1   text         1050 non-null   object
 2   chunks       1050 non-null   object
 3   chunk_count  1050 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 32.9+ KB


#### Step 2. Prompts

Writing the prompt is a crucial step in using LLMs. The prompt is the input to the LLM, and it is the only way to control the output of the LLM. In this step, we will be writing the prompt for the chat completion task. This process is often iterative, relying on the output of the LLM to refine the prompt. The iterative nature of the prompting process gives it the name **prompt engineering**.

In [51]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage
# Define the base prompt template
system_prompt = """You are an expert in analyzing political speeches for populist content. 
You can analyze text in any language."""

human_prompt = """Your task is to evaluate the level of populism in this political text:

{text}

A populist text must contain BOTH of these elements:

1. People-centrism:
- Focus on "the people" or "ordinary people" as an indivisible/homogeneous community
- Promotes politics as the popular will of "the people"
- NOTE: Appeals to specific subgroups (ethnicities, regional groups, classes) are NOT populist

2. Anti-elitism:
- Focus on "the elite" with negative descriptions
- Presents elite vs people as a moral struggle between good and bad
- NOTE: Criticism of specific elite members is NOT populist - must reject elite as a whole

Rate from 0-2:
0 = Not populist
1 = Somewhat populist
2 = Highly populist

Respond with: [score]; [brief justification]"""

# Create the chat prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", human_prompt),
])

### Step 3. Chains

What is a chain?

A chain is a sequence of steps that are executed in order. In the context of LangChain, a chain is a sequence of components or actions linked together to process inputs and produce desired outputs. For example, a chain might involve retrieving data, processing it through an LLM, and then formatting the output.


Let's first test the prompt with a single chunk.

In [52]:
test_chunk = df_clean.iloc[0]['chunks'][0]

In [53]:
from langchain import LLMChain
from langchain.schema.output_parser import StrOutputParser  

# Create the chain
chain = prompt_template | llm | StrOutputParser()

# Test prompt with single chunk
try:
    # Use the chain we created earlier
    response = chain.invoke({"text": test_chunk})
    
    print("Input chunk preview:")
    print(test_chunk[:300], "...\n")
    
    print("LLM Response:")
    score, justification = response.split(';', 1)
    print(f"Score: {score.strip()}")
    print(f"Justification: {justification.strip()}")
    
except Exception as e:
    print(f"Error processing text: {str(e)}")

Input chunk preview:
Primer Año del Gobierno del Pueblo
Primer Aniversario del Gobierno de 
Reconciliación y Unidad Nacional, 
Gobierno del Poder Ciudadano 
Plaza de la Revolución

10 de enero del 2007 


“Quiero, ante ustedes, citar nuevamente al Papa Benedicto XVI. Juan Pablo decía: capitalismo salvaje, los pueblos no ...

LLM Response:
Score: 2
Justification: The text contains strong elements of people-centrism by repeatedly referring to "el pueblo" and emphasizing the importance of the people's power and participation in decision-making. It also includes anti-elitism by criticizing the capitalist system and highlighting the need to transform global development models that benefit a minority elite. The speech frames the struggle as one between the people and the elite, portraying the elite as responsible for societal injustices.


Now we apply it to all the speeches in the dataframe if we are satisfied with the prompt.

In [54]:
# Create the chain
# chain = prompt_template | llm | StrOutputParser()

# Process all rows in the dataframe
results = []

for index, row in df_clean.iterrows():
    chunk_scores = []
    chunk_justifications = []
    
    for chunk_idx, chunk in enumerate(row['chunks']):
        try:
            response = chain.invoke({"text": chunk})
            
            if ';' in response:
                score_part, justification = response.split(';', 1)
                score_str = score_part.replace('Score:', '').strip()
                score = float(score_str)
                chunk_scores.append(score)
                chunk_justifications.append(justification.strip())
            
            time.sleep(1)
            
        except Exception as e:
            # Only print when there's an error
            print(f"\nError in Speech: {row['speech_name']}")
            print(f"Chunk {chunk_idx}")
            print(f"Response causing error: {response}")
            print(f"Error type: {type(e).__name__}")
            continue
    
    if chunk_scores:
        avg_score = sum(chunk_scores) / len(chunk_scores)
        results.append({
            'speech_name': row['speech_name'],
            'populism_score': round(avg_score, 2),
            'num_chunks_processed': len(chunk_scores),
            'justifications': chunk_justifications
        })


Error in Speech: Australia_Morrison_Ribbon_1
Chunk 0
Response causing error: [score]; The speech by the Prime Minister of Australia does not exhibit clear elements of populism. While it may contain some elements of people-centrism by honoring the sacrifices of Australian soldiers, it does not display anti-elitism or frame politics as a struggle between the elite and the people.
Error type: ValueError

Error in Speech: Colombia_Santos_International_1
Chunk 0
Response causing error: [score]; [brief justification]
Error type: ValueError


In [55]:
# Create final results dataframe
results_df = pd.DataFrame(results)

# Display summary
print(f"Processed {len(results_df)} speeches")
print("\nFirst few results:")
print(results_df.head())

Processed 1044 speeches

First few results:
                        speech_name  populism_score  num_chunks_processed  \
0         Nicaragua_Ortega_Famous_1             2.0                     2   
1            France_Chirac_Famous_1             2.0                     1   
2             Serbia_Tadic_Famous_1             2.0                     1   
3                 UK_Blair_Ribbon_3             2.0                     1   
4  Malaysia_Mohamad_International_1             1.0                     1   

                                      justifications  
0  [The text focuses heavily on "the people" and ...  
1  [This text contains strong elements of populis...  
2  [The text exhibits a high level of populism as...  
3  [This political text contains a strong emphasi...  
4  [The speech does contain elements of populism,...  


In [56]:
# Save to CSV
results_df.to_csv('data/populism_analysis_results4.csv', index=False)

#### Step 4. Validation

According to Törnberg (2023): 

"The Krippendorff’s alpha gives a measure of interrater agreement, and is used to assess the extent to which multiple raters or coders agree when coding or categorizing qualitative data. Krippendorff’s alpha takes into account both the observed agreement among raters and the expected agreement by chance. It can be applied to different types of nominal, ordinal, or interval-level data."

Hypothetically, if we have the 'ground truth' data provided by human annotators, we are able to use the Krippendorff's alpha for reliability check (illustration purpose only in the example below).

In [57]:
import numpy as np
from krippendorff import alpha

# First, create a mapping of speech names to their indices in the original df
original_speech_map = {name: idx for idx, name in enumerate(df['speech_name'])}

# Create random ground truth scores (0, 1, or 2) for the original dataframe
np.random.seed(42)  # for reproducibility
df['ground_truth'] = np.random.choice([0, 1, 2], size=len(df))

# Create a merged dataframe that includes both LLM scores and ground truth
merged_df = pd.merge(
    results_df,
    df[['speech_name', 'ground_truth']],
    on='speech_name',
    how='inner'
)

print("Number of speeches in original df:", len(df_clean))
print("Number of speeches in results_df:", len(results_df))
print("Number of matched speeches:", len(merged_df))


Number of speeches in original df: 1050
Number of speeches in results_df: 1044
Number of matched speeches: 1044


Out of curiosity, let's see if there are any speeches that weren't processed by the model.

In [58]:
# Find speeches that weren't processed by the model
unprocessed_df = df[~df['speech_name'].isin(results_df['speech_name'])]

print("Analysis of missing annotations:")
print(f"Total speeches in original dataset: {len(df)}")
print(f"Speeches processed by model: {len(results_df)}")
print(f"Speeches missing annotations: {len(unprocessed_df)}")

# Display some details about the unprocessed speeches
if len(unprocessed_df) > 0:
    print("\nSample of unprocessed speeches:")
    print(unprocessed_df[['speech_name', 'chunks']].head())

Analysis of missing annotations:
Total speeches in original dataset: 1161
Speeches processed by model: 1044
Speeches missing annotations: 117

Sample of unprocessed speeches:
                               speech_name chunks
3   Georgia_Margvelashvili_International_1     []
15          Kazakhstan_Nazarbayev_Ribbon_2     []
24           Estonia_Ansip_International_3     []
27           Estonia_Ansip_International_2     []
37        Bulgaria_Borisov_International_1     []


Why are these speeches not processed by the model?

In [59]:
# Examine the original text of speeches with empty chunks
print("Sample of speeches with empty chunks:")
for idx, row in unprocessed_df.head().iterrows():
    speech_name = row['speech_name']
    original_text = df[df['speech_name'] == speech_name]['text'].values[0]
    print(f"\n{speech_name}:")
    print(f"First 200 characters: {original_text[:200]}")
    print(f"Text length: {len(original_text)}")
    print(f"Is text empty? {len(original_text.strip()) == 0}")

Sample of speeches with empty chunks:

Georgia_Margvelashvili_International_1:
First 200 characters: 
Text length: 0
Is text empty? True

Kazakhstan_Nazarbayev_Ribbon_2:
First 200 characters: 
Text length: 0
Is text empty? True

Estonia_Ansip_International_3:
First 200 characters: 
Text length: 0
Is text empty? True

Estonia_Ansip_International_2:
First 200 characters: 
Text length: 0
Is text empty? True

Bulgaria_Borisov_International_1:
First 200 characters: 
Text length: 0
Is text empty? True


The empty text points to the need of possibly improving the auto-encoding function as well as how we cleaned empty chunks in above steps (note the possible difference between an empty chunk list and an empty chunk).

In the below code, we will calculate the Krippendorff's alpha to check the reliability of the model (illustration purpose only in our example).

In [60]:
# Prepare data for Krippendorff's alpha
# Convert to matrix where each row is a speech and each column is a rater
reliability_data = np.array([
    merged_df['populism_score'].values,
    merged_df['ground_truth'].values
])

# Calculate Krippendorff's alpha
k_alpha = alpha(reliability_data=reliability_data.T, level_of_measurement='ordinal')

print("\nKrippendorff's alpha:", k_alpha)

# Basic agreement statistics
agreement_df = merged_df.copy()
agreement_df['exact_match'] = agreement_df['populism_score'] == agreement_df['ground_truth']
agreement_df['diff'] = abs(agreement_df['populism_score'] - agreement_df['ground_truth'])

print("\nAgreement Statistics:")
print(f"Exact matches: {agreement_df['exact_match'].mean():.2%}")
print(f"Average difference: {agreement_df['diff'].mean():.2f}")

# Distribution of scores
print("\nScore Distribution:")
print("LLM Scores:")
print(merged_df['populism_score'].value_counts().sort_index())
print("\nGround Truth:")
print(merged_df['ground_truth'].value_counts().sort_index())

# Save the merged results
merged_df.to_csv('data/populism_analysis_with_ground_truth4.csv', index=False)


Krippendorff's alpha: 0.14489115221108617

Agreement Statistics:
Exact matches: 31.23%
Average difference: 0.90

Score Distribution:
LLM Scores:
populism_score
0.00     27
0.75      1
1.00    358
1.25      1
1.33      2
1.40      2
1.50     23
1.60      2
1.67      7
1.75      2
1.83      1
2.00    618
Name: count, dtype: int64

Ground Truth:
ground_truth
0    370
1    337
2    337
Name: count, dtype: int64


#### Model Selection

Cova and Schmitz (2024) recommend two studies on LLM model selection:

Törnberg's six principles for model selection:

- Reproducibility
- Ethnic & legality
- Transparency
- Culture and Language
- Scalability
- Complexity

The teacher-student model proposed by Weber and Reichardt (2024):

<img src="image.png" width="400" height="400">

### Case 2: Embeddings, RAG, and Vector Stores

#### What is word embedding

In the previous workshop, we learned about text representations where every document is represented by a (weighted) sum of the words it contains. In these models, the reprentation of a word is a **one-hot encoding** - a vector with one dimension per unique word in the vocabulary containing a 1 if the word is present in the document and 0 otherwise.

Example: we represent a word, *cat*, using the one-hot encoding reprentation with a vector of length the size of the vocabulary *J*, where the dimension corresponding to cat is 1 and all other dimensions are 0.

cat = (0,0,0,1,0,0,...0)

A consequence of this representation is that every word is treated uniquely, and the similarity between words is not captured. For example, *cat* and *dog* are completely different words, and thus represented by completely different vectors.

However, in reality we know that many words have highly similar meanings. The distributed representations covered in this chapter generalize this idea by learning from external data the semantic relationships between words like *writer* and *author* even though they don't share a common stem. 

With the above example, we will instead represent the word *cat* by using data to estimate a dense vector of length *K*, where *K < J*:

cat = (1.3, 0.2,...,0.56).

Because these techniques embed words into a common low-dimensional space (relative to the size of the vocabulary), they are called **word embeddings**.

#### What is RAG?

Retrieval-Augmented Generation (RAG) is a cutting-edge approach in natural language processing that combines document retrieval and generative modeling to create contextually aware and accurate responses. It relies heavily on word embeddings to connect these two steps: retrieval and generation.

**Retrieval with Word Embeddings**

The retrieval phase identifies relevant documents or knowledge snippets from a large external database or corpus. Here’s how word embeddings come into play:

- Encoding Query and Documents: The input query and all documents in the corpus are converted into vector representations using word embedding techniques (e.g., sentence embeddings, trained using models like BERT or Sentence Transformers). These embeddings map textual data into a shared high-dimensional semantic space.
	
- Similarity Search: The query embedding is compared against the document embeddings using similarity metrics such as cosine similarity. This enables RAG to retrieve documents not only based on exact keyword matches but also on semantic meaning. For example, a query about “climate change effects” might retrieve documents discussing “global warming impacts” due to their semantic proximity.

**Generation with Retrieved Context**

Once the most relevant documents are retrieved, they serve as additional context for the generative model. This process unfolds as follows:

- Incorporating Context: The retrieved documents are concatenated with the input query or transformed into prompts for the generative model. This gives the model access to external, up-to-date, or domain-specific information that complements its pre-trained knowledge.
	
- Text Generation: Using the combined input, the generative model produces a coherent, informed, and contextually relevant response. For instance, if the retrieved document explains “reducing stress through mindfulness,” the model might generate an answer to a query about meditation benefits that incorporates this information.

Example:

If a user asks, “What are the benefits of renewable energy?”:

- The retrieval phase uses embeddings to locate documents discussing renewable energy and its impacts, such as reducing carbon emissions or promoting sustainability.

- The generation phase takes this retrieved data and produces a response like: “Renewable energy offers numerous benefits, including reducing greenhouse gas emissions, decreasing reliance on fossil fuels, and supporting environmental sustainability.”

#### Differences between RAG and Fine-Tuning

RAG and fine-tuning are two distinct methods for enhancing the performance of generative models in natural language processing, each with its own approach to leveraging knowledge and improving accuracy.

**Fine-Tune**

Fine-tuning involves adapting a pre-trained model to a specific task or domain by updating its weights using labeled training data. This process essentially embeds task-specific knowledge directly into the model. Here’s how it works:

- Data Requirements: Requires a dataset of input-output pairs relevant to the target task or domain (e.g., medical summaries, legal text generation).

- Training Process: The model is trained on this data by adjusting its internal parameters, essentially “learning” the patterns and specific nuances of the target task.

- Output: After fine-tuning, the model is self-contained, capable of generating context-aware responses without needing access to external knowledge.

Limitations: Fine-tuning is resource-intensive, requiring large amounts of labeled data and computational power. It also “locks in” knowledge from the training set, making it difficult to update the model with new information.

**RAG**

RAG, in contrast, bypasses the need to encode task-specific knowledge into the model itself. Instead, it integrates a retrieval step to dynamically fetch relevant information from an external database or knowledge corpus at runtime. Here’s how it differs:

- Data Requirements: Relies on a knowledge base (e.g., a document corpus, Wikipedia) rather than labeled fine-tuning data.

- Process: explained above.
    
- Output: RAG’s responses are informed by up-to-date or domain-specific knowledge without altering the underlying model.

Advantages: RAG is flexible and scalable, allowing the knowledge base to be updated independently of the model. It is also less resource-intensive as it doesn’t require retraining the model for every new task or domain.

Example:

For a task like answering questions about current events:

- Fine-Tuning: The model would need to be retrained with labeled examples of questions and answers about current events.
	
- RAG: The model dynamically retrieves recent articles about the event and generates a response using that information.


#### How to implement RAG?

In this example below, we will be using the `LangChain` library and the same model, gpt-3.5-turbo, to implement RAG. This section is adapted from the example introduced in the following video: [Building a RAG application from scratch using Python, LangChain, and the OpenAI API](https://www.youtube.com/watch?v=BrsocJb-fAo). We will be using the materials from this tutorial, a transcript of an interivew, to build a RAG application that can answer questions based on the content of the transcript.

#### Set up the environment and the model

In [61]:
# We've done this step in the previous section.
# Set up OpenAI API key and Pinecone API key
load_dotenv(find_dotenv())

# Initialize the LLM
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

# Test the LLM
print(llm.invoke("What is Artificial Intelligence?"))

Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, especially computer systems. These processes include learning, reasoning, problem-solving, perception, and language understanding. AI technologies are used to develop systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI is a rapidly evolving field with applications in various industries, including healthcare, finance, transportation, and entertainment.


##### Set up the prompt template

Note that in the below template, the context is the transcript of the interview, and the question is the user's query. These are the two input variables that we will be using to build our RAG application. We generate the prompt template first (without talking to the model at this stage).

In [62]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply with "I don't know."

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Mary's sister is Susana.", question="Who is Mary's sister?")

'Human: \nAnswer the question based on the context below. If you can\'t \nanswer the question, reply with "I don\'t know."\n\nContext: Mary\'s sister is Susana.\n\nQuestion: Who is Mary\'s sister?\n'

We then chain the prompt with our model and an output parser, `StrOutputParser()`, to parse the output into a string. Chain is a sequence of steps that are executed in order. 

In [63]:
from langchain.schema.output_parser import StrOutputParser

parser = StrOutputParser()

chain = prompt | llm | parser

chain.invoke({
    "context": "Mary's sister is Susana.", 
    "question": "Who is Mary's sister?"
})

'Susana'

##### Reading in the transcript as context

OpenAI's [whisper](https://github.com/openai/whisper) model is a speech-to-text model that can be used to transcribe audio files. Here, we will directly read in the transcript of the interview as the context.

In [64]:
with open("data/transcription.txt") as file:
    transcription = file.read()

transcription[:100]

"I think it's possible that physics has exploits and we should be trying to find them. arranging some"

Can we use the entire transcript as the context?

In [65]:
try:
    chain.invoke({
        "context": transcription,
        "question": "Is reading papers a good idea?"
    })
except Exception as e:
    print(e)

This model's maximum context length is 16385 tokens. However, your messages resulted in 47050 tokens. Please reduce the length of the messages.


Again, we will be splitting the context.

In [66]:
# loading the transcript in memory
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/transcription.txt")
text_documents = loader.load()

In [67]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

Here is how embeddings come into play. When we pass the question, an embedding model encodes the question and the chunked context to find the most relevant chunks by calculating the similarity between the question and the chunks. We can then select the chunks with the highest similarity to the question, and use them as the context for the model. 

Let's play it out with the short sentences example above.

In [68]:
# Note: here I am using an old version of OpenAI (0.28.1) because of my python version
# Syntax might be different for openai version 1.0.0 and above.

from langchain_community.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

embedded_query = embeddings.embed_query("Who is Mary's sister?")

print(f"Embedding length: {len(embedded_query)}")
print(embedded_query[:10])

Embedding length: 1536
[-0.0013799423062174738, -0.034500904821185445, -0.011502386217451244, 0.0012415571995061765, -0.026119610627742672, 0.009081818279194359, -0.01564924997673654, 0.001727859744481776, -0.011827630141686142, -0.03319992539895551]


We can now generate the embeddings of the below sentences, and calculate the cosine similarity between the query and the sentences.

In [69]:
sentence1 = embeddings.embed_query("Mary's sister is Susana")
sentence2 = embeddings.embed_query("Pedro's mother is a teacher")

In [70]:
from sklearn.metrics.pairwise import cosine_similarity

query_sentence1_similarity = cosine_similarity([embedded_query], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([embedded_query], [sentence2])[0][0]

query_sentence1_similarity, query_sentence2_similarity

(0.9173235346162653, 0.7679756802174776)

##### What is a Vector Store?

A vectorstore is a specialized data storage system designed to store, index, and retrieve vectors efficiently. In the context of machine learning and NLP, vectors are numerical representations of data, such as word embeddings, document embeddings, or feature representations of any data point. Vectorstores are especially useful for similarity search, clustering, and other operations where comparisons of high-dimensional data are required.

To put it simply, a vectorstore is a database of embeddings, and specifically designed for similarity search on the vectors.

In [71]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore1 = DocArrayInMemorySearch.from_texts(
    [
        "Mary's sister is Susana",
        "John and Tommy are brothers",
        "Patricia likes white cars",
        "Pedro's mother is a teacher",
        "Lucia drives an Audi",
        "Mary has two siblings",
    ],
    embedding=embeddings,
)

In [72]:
vectorstore1.similarity_search_with_score(query="Who is Mary's sister?", k=3)

[(Document(page_content="Mary's sister is Susana"), 0.9173235428839119),
 (Document(page_content='Mary has two siblings'), 0.9045029978848249),
 (Document(page_content='John and Tommy are brothers'), 0.8013182122337668)]

After understanding the example above, we can now connect the vector store with a chain. To achieve this, we need to configure a [retriever](https://python.langchain.com/docs/how_to/#retrievers) in addition to the chain we created in the previous section.



In [73]:
retriever1 = vectorstore1.as_retriever()
retriever1.invoke("Who is Mary's sister?")

[Document(page_content="Mary's sister is Susana"),
 Document(page_content='Mary has two siblings'),
 Document(page_content='John and Tommy are brothers'),
 Document(page_content="Pedro's mother is a teacher")]

In [74]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever1, question=RunnablePassthrough())
setup.invoke("What color is Patricia's car?")

{'context': [Document(page_content='Patricia likes white cars'),
  Document(page_content='Lucia drives an Audi'),
  Document(page_content="Pedro's mother is a teacher"),
  Document(page_content="Mary's sister is Susana")],
 'question': "What color is Patricia's car?"}

In [75]:
chain = setup | prompt | llm | parser
chain.invoke("What color is Patricia's car?")

'White'

In [76]:
chain = setup | prompt | llm | parser
chain.invoke("What car does Lucia drive?")

'Audi'

Now let's create a new vector store with the chunks of the transcript. 

In [77]:
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

Let's set up a new chain using the new vector store. This time we are using a different equivalent syntax to specify the RunnableParallel portion of the chain. We then use `chain.invoke()` to generate the response.

In [78]:
chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)
chain.invoke("What is synthetic intelligence?")

'Synthetic intelligence is described as the next stage of development, potentially uncovering and solving the puzzle of the universe. It is suggested that synthetic AIs will be able to generate art, ideas, and emotions in an automated way.'

##### Performance Evaluation

How do we evaluate the performance of LLMs on a RAG task?

Evaluating an LLM on RAG requires measuring both the retrieval effectiveness and genration quality. In the process of evaluating retrieval effectiveness, it is recommended to use multiple retrieval metrics (e.g., precision, recall, F1 scores). For an evaluation of the generation quality, we can look into factual correctness, relevance, and coherence of the generated response. A combination of automated approaches and human evaluation is always recommended.




# References

- Modeling Human Behavior Computationally
	- Aher, Gati, Rosa I. Arriaga, and Adam Tauman Kalai. (2022). Using Large Language Models to Simulate Multiple Humans and Replicate Human Subject Studies. [arXiv](https://doi.org/10.48550/ARXIV.2208.10264).
	-	Argyle, Lisa P., Ethan C. Busby, Nancy Fulda, Joshua R. Gubler, Christopher Rytting, and David Wingate. (2023). Out of One, Many: Using Language Models to Simulate Human Samples. Political Analysis, 31(3), 337–51. [link](https://doi.org/10.1017/pan.2023.2).
	-	Dillion, Danica, Niket Tandon, Yuling Gu, and Kurt Gray. (2023). Can AI Language Models Replace Human Participants? Trends in Cognitive Sciences, 27(7), 597–600. [link](https://doi.org/10.1016/j.tics.2023.04.008).
    -	Jiang, Ferrara. (2023). Social-LLM: Modeling User Behavior at Scale using Language Models and Social Network Data. [arXiv](https://arxiv.org/abs/2401.00893?utm_source=chatgpt.com)
	-   Tjuatja, Lindia, Valerie Chen, Sherry Tongshuang Wu, Ameet Talwalkar, and Graham Neubig.“Do LLMs Exhibit Human-Like Response Biases? A Case Study in Survey Design.” [arXiv](https://doi.org/10.48550/ARXIV.2311.04076)


- Simulating Social Relationships
	-	Akata, Elif, Lion Schulz, Julian Coda-Forno, Seong Joon Oh, Matthias Bethge, and Eric Schulz. (2023). Playing Repeated Games with Large Language Models. [arXiv](https://doi.org/10.48550/ARXIV.2305.16867).
	-	Park, Joon Sung, Joseph C. O’Brien, Carrie J. Cai, Meredith Ringel Morris, Percy Liang, and Michael S. Bernstein. (2023). Generative Agents: Interactive Simulacra of Human Behavior. [arXiv](https://arxiv.org/abs/2304.03442).
	-	Wang, Lei, Chen Ma, Xueyang Zeng, Zeyu Zhang, Hao Yang, Jingsen Zhang, Zhiyuan Chen, et al. (2024). A Survey on Large Language Model Based Autonomous Agents. Frontiers of Computer Science, 16(8), 186345. [link](https://doi.org/10.1007/s11704-024-40231-1).


- Interacting with Human Agents
	-	Chopra, Felix, and Ingar Haaland. (2023). Conducting Qualitative Interviews with AI. SSRN Electronic Journal. [link](https://doi.org/10.2139/ssrn.4583756).


- Text Annotation
	-	Gilardi, Fabrizio, Meysam Alizadeh, and Maël Kubli. (2023). ChatGPT Outperforms Crowd Workers for Text-Annotation Tasks. Proceedings of the National Academy of Sciences, 120(30), e2305016120. [link](https://doi.org/10.1073/pnas.2305016120).	
	-	Laurer, Moritz. (2024). Synthetic Data: Save Money, Time and Carbon with Open Source. Hugging Face Blog. [link](https://huggingface.co/blog/synthetic-data-save-costs).
	-	Leek, Lauren Caroline, Simon Bischl, and Maximilian Freier. (2024). Introducing Textual Measures of Central Bank Policy-Linkages Using ChatGPT. [link](https://doi.org/10.31235/osf.io/78wnp).
	-	Törnberg, Petter. (2023). ChatGPT-4 Outperforms Experts and Crowd Workers in Annotating Political Twitter Messages with Zero-Shot Learning. [arXiv](https://doi.org/10.48550/ARXIV.2304.06588).


- Steamlining Research Pipeline
	-	Korinek. (2023). Generative AI for Economic Research: Use Cases and Implications for Economists. Journal of Economic Literature, 61(4), 1281-1317. [link](https://www.aeaweb.org/articles?id=10.1257/jel.20231736)


- Model Selection
    -   Törnberg, Petter.(2024). Best Practices for Text Annotation with Large Language Models. Sociologica, 18(2).[link](https://sociologica.unibo.it/article/view/19461/18663)
    -   Weber and Reichardt (2024). Evaluation is All You Need. Prompting Generative Large Language Models for Annotation Tasks in the Social Sciences. A Primer Using Open Models. [arXiv](https://arxiv.org/abs/2401.00284)
